In [25]:
"""
A&P
Finance data SAP

Filtrer Managerial EPM for USA
Extract brands from CODE EPM
ADVERTISING from R4100
PROMO from R4200


If faut columns pour:
	année, mois, brand, A, P

Groupby brand / sort by Date
"""

'\nA&P\nFinance data SAP\n\nFiltrer Managerial EPM for USA\nExtract brands from CODE EPM\nADVERTISING from R4100\nPROMO from R4200\n\n\nIf faut columns pour:\n\tannée, mois, brand, A, P\n\nGroupby brand / sort by Date\n'

In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from calendar import Calendar

In [179]:
df_AP = pd.read_excel('data/Growth Modelling - USA - 2018-2021 - Finance Data (SAP - Flux Rio).xlsx', header=17)

In [180]:
df_USA = df_AP[df_AP['MANAGERIAL EPM'] == 'USA']

In [181]:
#On vire Kiri pour les US , '000KE - KIRI'
bel_brands = ['000BQ - BOURSIN', '000LA - THE LAUGHING COW', '000MB - MINI BABYBEL', '000KC - KAUKAUNA', "000PG - PRICE'S", '000MI - MERKTS', '003NH - NURISHH']
df_bel_brands = df_USA[df_USA['CODE EPM'].isin(bel_brands)]

In [182]:
df_final = df_bel_brands[['YEAR EPM', 'CODE EPM', 'R4100 - ADVERTISING', 'R4200 - PROMOTION - CONSUMERS']]

In [183]:
df_final = df_final.rename(columns={'YEAR EPM':'Year', 'CODE EPM':'Brand', 'R4100 - ADVERTISING':'Advertising', 'R4200 - PROMOTION - CONSUMERS':'Promotion'})

In [184]:
df_final['Advertising'] = df_final['Advertising'].abs()
df_final['Promotion'] = df_final['Promotion'].abs()

In [185]:
df_final['Brand'] = df_final['Brand'].apply(lambda x:x.split(sep='-')[-1].strip())

In [186]:
df_final['Month'] = df_final['Year'].apply(lambda x:int(x[5:8]))

In [187]:
df_final['Year'] = df_final['Year'].apply(lambda x:int(x[:4]))

In [188]:
df_final = df_final.fillna(0.0)

In [189]:
def count_num_sundays_in_month(year, month):
    day_to_count = calendar.SUNDAY
    matrix = calendar.monthcalendar(year, month)
    num_days = sum(1 for x in matrix if x[day_to_count] != 0)
    return num_days

In [190]:
df_final['number of weeks'] = df_final.apply(lambda x:count_num_sundays_in_month(x.Year, x.Month), axis=1)

In [191]:
df_final['A&P'] = df_final.apply(lambda x: (x.Advertising + x.Promotion) / x['number of weeks'] * 1000, axis=1)

In [192]:
df_final

,Year,Brand,Advertising,Promotion,Month,number of weeks,A&P
818,2018,BOURSIN,80.337033,40.643074,1,4,30245.026650
824,2018,THE LAUGHING COW,602.204522,152.581049,1,4,188696.392850
829,2018,MINI BABYBEL,1538.474628,416.037598,1,4,488628.056575
836,2018,KAUKAUNA,1.147541,29.623066,1,4,7692.651650
839,2018,PRICE'S,0.196008,23.975959,1,4,6042.991800
...,...,...,...,...,...,...,...
80746,2021,MINI BABYBEL,952.971199,539.338324,12,4,373077.380850
80758,2021,KAUKAUNA,13.314860,23.189343,12,4,9126.050925
80764,2021,PRICE'S,13.230278,77.810720,12,4,22760.249325
80824,2021,MERKTS,0.309191,62.009957,12,4,15579.787225


In [193]:
full_idx = pd.date_range(start='2017-12-31', end='2021-12-26', freq='W')
df_test = pd.DataFrame(index=full_idx)

df_test['Year'] = df_test.index.year
df_test['Month'] = df_test.index.month

In [194]:
df_concat = pd.DataFrame()
for brand in df_final.Brand.unique():
    df_concat = pd.concat([df_concat, pd.merge(df_final[df_final.Brand==brand], df_test.reset_index(), on=['Year', 'Month']).rename(columns={'index':'Date'})])

In [195]:
df_concat[['Date', 'Brand', 'A&P']]

,Date,Brand,A&P
0,2018-01-07,BOURSIN,30245.026650
1,2018-01-14,BOURSIN,30245.026650
2,2018-01-21,BOURSIN,30245.026650
3,2018-01-28,BOURSIN,30245.026650
4,2018-02-04,BOURSIN,25076.470000
...,...,...,...
47,2021-11-28,NURISHH,24981.746625
48,2021-12-05,NURISHH,62817.249700
49,2021-12-12,NURISHH,62817.249700
50,2021-12-19,NURISHH,62817.249700


In [196]:
df_concat.Brand.unique()

array(['BOURSIN', 'THE LAUGHING COW', 'MINI BABYBEL', 'KAUKAUNA',
       "PRICE'S", 'MERKTS', 'NURISHH'], dtype=object)

In [200]:
df_concat.groupby('Brand').count()

,Year,Advertising,Promotion,Month,number of weeks,A&P,Date
Brand,,,,,,,
BOURSIN,208,208,208,208,208,208,208
KAUKAUNA,208,208,208,208,208,208,208
MERKTS,208,208,208,208,208,208,208
MINI BABYBEL,208,208,208,208,208,208,208
NURISHH,52,52,52,52,52,52,52
PRICE'S,208,208,208,208,208,208,208
THE LAUGHING COW,208,208,208,208,208,208,208
